In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.abspath('../../app'))

In [3]:
from core.feluda import Feluda,ComponentType

In [4]:
feluda = Feluda("config.yml")
feluda.setup()

Installing packages for vid_vec_rep_clip


/usr/app/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/app/venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
if feluda.config.store:
    feluda.start_component(ComponentType.STORE)

In [6]:
# self.config.operators.parameters[0].type

In [7]:
url = "https://github.com/aatmanvaidya/audio-files/raw/main/feluda-media/en-speech.mp4"

feluda.store_video(url)

100% [..............................................................................] 48488 / 48488
Video downloaded
----> 6 (2, [])


('result:', {'message': 'multiple media stored'})

In [8]:
feluda.search_video(url)

100% [..............................................................................] 48488 / 48488
Video downloaded
calculation: 1 / (1 + l2norm(params.query_vector, 'vid_vec'))


[{'doc_id': 'TgLwrZIBZCwUx_cLbhBV',
  'dist': 0.9999998,
  'dataset': 'en-speech.mp4',
  'e_kosh_id': '',
  'text': None,
  'metadata': None},
 {'doc_id': 'TwLwrZIBZCwUx_cLbhBX',
  'dist': 0.9999998,
  'dataset': 'en-speech.mp4',
  'e_kosh_id': '',
  'text': None,
  'metadata': None},
 {'doc_id': 'UALyrZIBZCwUx_cL9hB2',
  'dist': 0.9999998,
  'dataset': 'en-speech.mp4',
  'e_kosh_id': '',
  'text': None,
  'metadata': None},
 {'doc_id': 'UQLyrZIBZCwUx_cL9hB2',
  'dist': 0.9999998,
  'dataset': 'en-speech.mp4',
  'e_kosh_id': '',
  'text': None,
  'metadata': None},
 {'doc_id': 'UgILrpIBZCwUx_cLPRAI',
  'dist': 0.9999998,
  'dataset': 'en-speech.mp4',
  'e_kosh_id': '',
  'text': None,
  'metadata': None},
 {'doc_id': 'UwILrpIBZCwUx_cLPRAI',
  'dist': 0.9999998,
  'dataset': 'en-speech.mp4',
  'e_kosh_id': '',
  'text': None,
  'metadata': None}]